In [1]:
import pandas as pd
import numpy as np
import re
import unicodedata

In [2]:
data_2013_2019 = pd.read_excel('/kaggle/input/leche/series-historicas-precios-mayoristas-leche-2013-2019.xlsx', header=None)
data_2020 = pd.read_excel('/kaggle/input/leche/series-historicas-precios-mayoristas-leche-2020.xlsx', header=None)
data_2021 = pd.read_excel('/kaggle/input/leche/series-historicas-precios-mayoristas-leche-2021.xlsx', header=None)
data_2022 = pd.read_excel('/kaggle/input/leche/series-historicas-precios-mayoristas-leche-2022.xlsx', header=None)
data_2023 = pd.read_excel('/kaggle/input/leche/anex-SIPSALeche-SerieHistoricaPrecios-2023.xlsx', header=None)
data_2024 = pd.read_excel('/kaggle/input/leche/anex-SIPSALeche-SerieHistoricaPrecios-2024.xlsx', header=None)
ipc = pd.read_csv('/kaggle/input/indice-ipc/IPC_Indice_de_Precios_al_Consumidor_2013-2024', sep='\t', header=None)

In [3]:
#print(data_2013_2019.head())
#print(data_2013_2019.tail())

#print(data_2020.head())
#print(data_2020.tail())

#print(data_2021.head())
#print(data_2021.tail())

#print(data_2022.head())
#print(data_2022.tail())

#print(data_2023.head())
#print(data_2023.tail())

#print(data_2024.head())
#print(data_2024.tail())

In [4]:
def limpiar_encabezado(df):
    """
    Limpia encabezados y pies de página:
    - Detecta la fila del encabezado (Año/Mes/Periodo).
    - Si la fila de encabezado o la siguiente tienen ANY NaN -> combina ambas filas como header.
    - Si no, usa solo la fila encontrada como header.
    - Después recorta el final por filas no válidas y elimina footers típicos.
    """
    df_copy = df.copy()

    # 1) Encontrar la fila del encabezado real
    header_candidates = df_copy[df_copy.iloc[:, 0].astype(str).str.contains('Año|Mes|Periodo', case=False, na=False)]
    if header_candidates.empty:
        raise ValueError("No se encontró una fila de encabezado válida ('Año', 'Mes' o 'Periodo').")
    header_row = header_candidates.index[0]

    # 2) Determinar si combinar encabezados (si header_row o header_row+1 tienen algún NaN)
    # Asegurarse de que exista header_row+1
    n_rows = df_copy.shape[0]
    row_has_nan = df_copy.iloc[header_row].isna().any()
    next_row_has_nan = False
    if header_row + 1 < n_rows:
        next_row_has_nan = df_copy.iloc[header_row + 1].isna().any()

    combine = row_has_nan or next_row_has_nan

    # 3) Aplicar la lógica: combinar o usar una sola fila como encabezado
    if combine and (header_row + 1 < n_rows):
        # combinar header_row y header_row+1
        combined_header = (df_copy.iloc[header_row].fillna('') + ' ' + df_copy.iloc[header_row + 1].fillna(''))
        combined_header = combined_header.astype(str).str.strip().replace('', np.nan)
        df_work = df_copy[header_row + 2:].copy()
        df_work.columns = combined_header
    else:
        # usar solo header_row (si la verdadera fila de nombres está en header_row)
        df_work = df_copy[header_row + 1:].copy()
        # si la primera fila tras header_row contiene los nombres, asignarla; si no, intenta usar header_row
        # aquí asumimos que header_row + 1 es la primera fila de datos, por eso tomamos header_row como nombres
        df_work.columns = df_copy.iloc[header_row].values
        # si accidentalmente la primera fila resultante es la fila de títulos (duplicada), eliminarla
        # comprobamos si la primera fila tiene valores iguales a las columnas -> si es así, eliminarla
        first_row_vals = df_work.iloc[0].astype(str).str.strip().tolist() if not df_work.empty else []
        col_names = [str(c).strip() for c in df_work.columns]
        if first_row_vals == col_names:
            df_work = df_work.iloc[1:].copy()

    # 4) Normalizar columnas y resetear índice
    df_work = df_work.reset_index(drop=True)
    df_work.columns = [str(c).strip() for c in df_work.columns]

    # 5) Recortar el final por la última fila válida en la primera columna
    primera_col = df_work.iloc[:, 0].astype(str).str.strip()
    pattern_valido = re.compile(r'^\d{4}$|^\d{1,2}$|^\d{4}[-/]\d{1,2}$')
    mask_valida = primera_col.apply(lambda x: bool(pattern_valido.match(x)) if pd.notna(x) and str(x).strip().lower() != 'nan' else False)
    if mask_valida.any():
        ultima_valida = mask_valida[mask_valida].index[-1]
        df_work = df_work.loc[:ultima_valida].copy()

    # 6) Eliminar filas completamente vacías
    df_work = df_work.dropna(how='all').reset_index(drop=True)

    # 7) Quitar footers típicos al final
    footer_patterns = ['Fuente', 'Actualiz', 'actualiz', 'n.d', 'no disponible', 'Tendencia']
    def fila_es_footer(row):
        if row.isna().all():
            return True
        first = str(row.iloc[0]).strip()
        if not pattern_valido.match(first):
            joined = ' '.join([str(x) for x in row.values if pd.notna(x)])
            if any(p.lower() in joined.lower() for p in footer_patterns):
                return True
            resto = row.iloc[1:]
            tiene_numerico = any(re.match(r'^-?\d+(\.\d+)?$', str(x).replace(',', '.').strip()) for x in resto.dropna().astype(str))
            if not tiene_numerico:
                return True
        return False

    while not df_work.empty and fila_es_footer(df_work.iloc[-1]):
        df_work = df_work.iloc[:-1].copy()

    df_work = df_work.reset_index(drop=True)
    return df_work

In [5]:
# Crear un diccionario con los DataFrames originales
archivos = {
    'data_2013_2019': data_2013_2019,
    'data_2020': data_2020,
    'data_2021': data_2021,
    'data_2022': data_2022,
    'data_2023': data_2023,
    'data_2024': data_2024
}

# Limpiar y mostrar estructura en un solo bucle
archivos_limpios = {}

for nombre, df in archivos.items():
    limpio = limpiar_encabezado(df)
    archivos_limpios[nombre] = limpio
    
    print(f"\n{nombre}:")
    print(f"Número de columnas: {len(limpio.columns)}")
    print("Nombres de columnas:")
    print(list(limpio.columns))


data_2013_2019:
Número de columnas: 7
Nombres de columnas:
['Año', 'Mes', 'Nombre departamento', 'Código departamento', 'Nombre municipio', 'Código municipio', 'Precio promedio por litro']

data_2020:
Número de columnas: 7
Nombres de columnas:
['Año', 'Mes', 'Nombre departamento', 'Código departamento', 'Nombre municipio', 'Código municipio', 'Precio promedio por litro']

data_2021:
Número de columnas: 6
Nombres de columnas:
['Mes y año', 'Nombre departamento', 'Código departamento', 'Nombre municipio', 'Código municipio', 'Precio promedio por litro']

data_2022:
Número de columnas: 6
Nombres de columnas:
['Mes y año', 'Nombre departamento', 'Código departamento', 'Nombre municipio', 'Código municipio', 'Precio promedio por litro']

data_2023:
Número de columnas: 6
Nombres de columnas:
['Mes y año', 'Nombre departamento', 'Código departamento', 'Nombre municipio', 'Código municipio', 'Precio promedio por litro']

data_2024:
Número de columnas: 6
Nombres de columnas:
['Mes y año', 'Nom

In [6]:
def estandarizar_columnas(df):
    """
    Asegura que todas las bases tengan las mismas 7 columnas:
    'Año', 'Mes', 'Nombre departamento', 'Código departamento',
    'Nombre municipio', 'Código municipio', 'Precio promedio por litro'.
    Si la columna 'Mes y año' existe, se separa en dos.
    """
    df = df.copy()
    
    # Caso: columna combinada "Mes y año"
    if 'Mes y año' in df.columns:
        # Intentar dividir en mes y año (asumiendo formato "2023-05", "05/2022", etc.)
        df[['Año', 'Mes']] = df['Mes y año'].astype(str).str.extract(r'(\d{4})[-/](\d{1,2})')
        # Si viene en formato "Mayo 2023" u otros, tratamos de separarlo
        if df['Año'].isna().any():
            df[['Mes', 'Año']] = df['Mes y año'].astype(str).str.extract(r'([A-Za-zñÑáéíóúÁÉÍÓÚ]+)\s*(\d{4})')
        df = df.drop(columns=['Mes y año'])
    
    # Reordenar columnas al formato estándar
    columnas_objetivo = [
        'Año',
        'Mes',
        'Nombre departamento',
        'Código departamento',
        'Nombre municipio',
        'Código municipio',
        'Precio promedio por litro'
    ]
    
    # Agregar las que falten como NaN
    for col in columnas_objetivo:
        if col not in df.columns:
            df[col] = pd.NA

    df = df[columnas_objetivo]
    return df

# Aplicar a todos los archivos limpios
archivos_estandarizados = {
    nombre: estandarizar_columnas(df) for nombre, df in archivos_limpios.items()
}

# Verificar resultado
for nombre, df in archivos_estandarizados.items():
    print(f"\n{nombre}: {len(df.columns)} columnas")
    print(df.columns.tolist())


data_2013_2019: 7 columnas
['Año', 'Mes', 'Nombre departamento', 'Código departamento', 'Nombre municipio', 'Código municipio', 'Precio promedio por litro']

data_2020: 7 columnas
['Año', 'Mes', 'Nombre departamento', 'Código departamento', 'Nombre municipio', 'Código municipio', 'Precio promedio por litro']

data_2021: 7 columnas
['Año', 'Mes', 'Nombre departamento', 'Código departamento', 'Nombre municipio', 'Código municipio', 'Precio promedio por litro']

data_2022: 7 columnas
['Año', 'Mes', 'Nombre departamento', 'Código departamento', 'Nombre municipio', 'Código municipio', 'Precio promedio por litro']

data_2023: 7 columnas
['Año', 'Mes', 'Nombre departamento', 'Código departamento', 'Nombre municipio', 'Código municipio', 'Precio promedio por litro']

data_2024: 7 columnas
['Año', 'Mes', 'Nombre departamento', 'Código departamento', 'Nombre municipio', 'Código municipio', 'Precio promedio por litro']


In [7]:
for nombre, df in archivos_estandarizados.items():
    print(f"\n{'='*80}")
    print(f"{nombre} — {len(df)} filas, {len(df.columns)} columnas")
    print(f"Columnas: {list(df.columns)}\n")
    print(df.head(5))  # Muestra las primeras 5 filas


data_2013_2019 — 16470 filas, 7 columnas
Columnas: ['Año', 'Mes', 'Nombre departamento', 'Código departamento', 'Nombre municipio', 'Código municipio', 'Precio promedio por litro']

    Año Mes Nombre departamento Código departamento Nombre municipio  \
0  2013  01           ANTIOQUIA                  05        ANGOSTURA   
1  2013  01           ANTIOQUIA                  05          BARBOSA   
2  2013  01           ANTIOQUIA                  05            BELLO   
3  2013  01           ANTIOQUIA                  05          BELMIRA   
4  2013  01           ANTIOQUIA                  05       DON MATIAS   

  Código municipio Precio promedio por litro  
0            05038                882.861045  
1            05079                931.337395  
2            05088                937.877513  
3            05086                 893.94587  
4            05237                854.095295  

data_2020 — 1456 filas, 7 columnas
Columnas: ['Año', 'Mes', 'Nombre departamento', 'Código departamen

In [8]:
def limpiar_texto(valor):
    """Convierte a minúsculas, quita tildes y espacios innecesarios."""
    if isinstance(valor, str):
        # Normalizar caracteres (quita tildes)
        valor = unicodedata.normalize('NFKD', valor).encode('ascii', 'ignore').decode('utf-8')
        return valor.strip().lower()
    return valor

def revisar_y_normalizar(df, nombre_df):
    """
    Revisa valores nulos y normaliza texto.
    Retorna el DataFrame limpio.
    """
    print(f"\n{'='*80}")
    print(f"{nombre_df} — Revisión de datos faltantes")
    print(df.isna().sum())  # Conteo de valores nulos
    
    # Aplicar limpieza a todas las celdas
    df_limpio = df.applymap(limpiar_texto)
    
    # Revisión rápida después de limpieza
    print("\nLimpieza completada (minúsculas y sin tildes)")
    print(df_limpio.head(3))
    
    return df_limpio

# Aplicar a todos los archivos estandarizados
archivos_normalizados = {
    nombre: revisar_y_normalizar(df, nombre)
    for nombre, df in archivos_estandarizados.items()
}



data_2013_2019 — Revisión de datos faltantes
Año                          0
Mes                          0
Nombre departamento          0
Código departamento          0
Nombre municipio             0
Código municipio             0
Precio promedio por litro    0
dtype: int64

Limpieza completada (minúsculas y sin tildes)
    Año Mes Nombre departamento Código departamento Nombre municipio  \
0  2013  01           antioquia                  05        angostura   
1  2013  01           antioquia                  05          barbosa   
2  2013  01           antioquia                  05            bello   

  Código municipio  Precio promedio por litro  
0            05038                 882.861045  
1            05079                 931.337395  
2            05088                 937.877513  

data_2020 — Revisión de datos faltantes
Año                          0
Mes                          0
Nombre departamento          0
Código departamento          0
Nombre municipio             0


/tmp/ipykernel_39/2504142068.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_limpio = df.applymap(limpiar_texto)
/tmp/ipykernel_39/2504142068.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_limpio = df.applymap(limpiar_texto)
/tmp/ipykernel_39/2504142068.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_limpio = df.applymap(limpiar_texto)
/tmp/ipykernel_39/2504142068.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_limpio = df.applymap(limpiar_texto)
/tmp/ipykernel_39/2504142068.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_limpio = df.applymap(limpiar_texto)
/tmp/ipykernel_39/2504142068.py:19: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_limpio = df.applymap(limpiar_texto)


In [9]:
# Copia del DataFrame original limpio
data_2021 = archivos_normalizados['data_2021'].copy()

# Asegurar que la columna de precios sea numérica
data_2021['Precio promedio por litro'] = pd.to_numeric(data_2021['Precio promedio por litro'], errors='coerce')

# Rellenar valores faltantes con el promedio por departamento
data_2021['Precio promedio por litro'] = (
    data_2021.groupby('Nombre departamento')['Precio promedio por litro']
    .transform(lambda x: x.fillna(x.mean()))
)

# Confirmar que ya no hay nulos
print("\n Verificación después del relleno:")
print(data_2021['Precio promedio por litro'].isna().sum())

# Reemplazar el DataFrame dentro del diccionario
archivos_normalizados['data_2021'] = data_2021



 Verificación después del relleno:
0


In [10]:
for nombre, df in archivos_normalizados.items():
    print(f"\n{'='*90}")
    print(f"📄 {nombre} — {len(df)} filas, {len(df.columns)} columnas")
    
    # Mostrar nombres de columnas
    print("\n Columnas:")
    print(list(df.columns))
    
    # Verificar datos faltantes
    print("\n Datos faltantes por columna:")
    print(df.isna().sum())
    
    # Tipos de datos
    print("\n Tipos de datos (dtype):")
    print(df.dtypes)
    
    # Muestra rápida de las primeras filas
    print("\n Vista previa de datos:")
    print(df.head(3))



📄 data_2013_2019 — 16470 filas, 7 columnas

 Columnas:
['Año', 'Mes', 'Nombre departamento', 'Código departamento', 'Nombre municipio', 'Código municipio', 'Precio promedio por litro']

 Datos faltantes por columna:
Año                          0
Mes                          0
Nombre departamento          0
Código departamento          0
Nombre municipio             0
Código municipio             0
Precio promedio por litro    0
dtype: int64

 Tipos de datos (dtype):
Año                            int64
Mes                           object
Nombre departamento           object
Código departamento           object
Nombre municipio              object
Código municipio              object
Precio promedio por litro    float64
dtype: object

 Vista previa de datos:
    Año Mes Nombre departamento Código departamento Nombre municipio  \
0  2013  01           antioquia                  05        angostura   
1  2013  01           antioquia                  05          barbosa   
2  2013  01  

In [11]:
# Definir columnas por tipo
cols_numericas = ['año', 'mes', 'precio promedio por litro']
cols_texto = ['nombre departamento', 'codigo departamento', 'nombre municipio', 'codigo municipio',]

# Aplicar conversión a todos los archivos
archivos_tipados = {}

for nombre, df in archivos_normalizados.items():
    df = df.copy()
    
    # Asegurar nombres en minúsculas (por si acaso)
    df.columns = [col.lower().strip() for col in df.columns]
    
    # Convertir columnas numéricas
    for col in cols_numericas:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce')  # convierte y pone NaN si falla
    
    # Convertir columnas de texto
    for col in cols_texto:
        if col in df.columns:
            df[col] = df[col].astype(str)
    
    archivos_tipados[nombre] = df

# Verificación de tipos
for nombre, df in archivos_tipados.items():
    print(f"\n{'='*90}")
    print(f" {nombre} — Tipos de datos después de estandarizar:\n")
    print(df.dtypes)



 data_2013_2019 — Tipos de datos después de estandarizar:

año                            int64
mes                            int64
nombre departamento           object
código departamento           object
nombre municipio              object
código municipio              object
precio promedio por litro    float64
dtype: object

 data_2020 — Tipos de datos después de estandarizar:

año                            int64
mes                            int64
nombre departamento           object
código departamento           object
nombre municipio              object
código municipio              object
precio promedio por litro    float64
dtype: object

 data_2021 — Tipos de datos después de estandarizar:

año                            int64
mes                            int64
nombre departamento           object
código departamento           object
nombre municipio              object
código municipio              object
precio promedio por litro    float64
dtype: object

 data_2022

In [12]:
import pandas as pd

# Aseguramos el orden correcto de concatenación (por año)
archivos_ordenados = [
    archivos_tipados['data_2013_2019'],
    archivos_tipados['data_2020'],
    archivos_tipados['data_2021'],
    archivos_tipados['data_2022'],
    archivos_tipados['data_2023'],
    archivos_tipados['data_2024']
]

# Unir todos los DataFrames
data_completa = pd.concat(archivos_ordenados, ignore_index=True)

# Verificación rápida
print(" Unión completada")
print("Filas totales:", len(data_completa))
print("Columnas:", data_completa.columns.tolist())
print("\nTipos de datos:")
print(data_completa.dtypes)
print("\nValores nulos por columna:")
print(data_completa.isna().sum())

# Guardar en CSV
data_completa.to_csv("precio_leche_2013_2024.csv", index=False, encoding='utf-8-sig')

print("\n Archivo guardado exitosamente como 'precio_leche_2013_2024.csv'")


 Unión completada
Filas totales: 27909
Columnas: ['año', 'mes', 'nombre departamento', 'código departamento', 'nombre municipio', 'código municipio', 'precio promedio por litro']

Tipos de datos:
año                            int64
mes                            int64
nombre departamento           object
código departamento           object
nombre municipio              object
código municipio              object
precio promedio por litro    float64
dtype: object

Valores nulos por columna:
año                          0
mes                          0
nombre departamento          0
código departamento          0
nombre municipio             0
código municipio             0
precio promedio por litro    0
dtype: int64

 Archivo guardado exitosamente como 'precio_leche_2013_2024.csv'


In [13]:
ipc.head()

,0,1
0,12/24,144.22
1,11/24,143.83
2,10/24,144.02
3,08/24,143.67
4,07/24,143.38


In [14]:
# Aseguramos nombres de columnas
ipc.columns = ['mes_año', 'ipc']

# Separar columna mes/año
ipc[['mes', 'año']] = ipc['mes_año'].str.split('/', expand=True)

# Convertir tipos
ipc['mes'] = ipc['mes'].astype(int)
ipc['año'] = 2000 + ipc['año'].astype(int)  # convierte 24→2024, 13→2013
ipc['ipc'] = pd.to_numeric(ipc['ipc'], errors='coerce')

# Seleccionar y ordenar
ipc = ipc[['año', 'mes', 'ipc']].sort_values(by=['año', 'mes']).reset_index(drop=True)

# Mostrar verificación
print(ipc.info())
print(ipc.head(12))
print(ipc.tail(12))

ipc.to_csv("ipc_2013_2024.csv", index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 138 entries, 0 to 137
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   año     138 non-null    int64  
 1   mes     138 non-null    int64  
 2   ipc     138 non-null    float64
dtypes: float64(1), int64(2)
memory usage: 3.4 KB
None
     año  mes       ipc
0   2013    1  112.1490
1   2013    2  112.6471
2   2013    3  112.8788
3   2013    4  113.1643
4   2013    5  113.4797
5   2013    6  113.7462
6   2013    7  113.7973
7   2013    8  113.8922
8   2013    9  114.2258
9   2013   10  113.9293
10  2013   11  113.6829
11  2013   12  113.9825
      año  mes     ipc
126  2023    8  134.45
127  2023    9  135.39
128  2023   10  136.11
129  2023   11  136.45
130  2023   12  137.09
131  2024    1  137.72
132  2024    6  142.92
133  2024    7  143.38
134  2024    8  143.67
135  2024   10  144.02
136  2024   11  143.83
137  2024   12  144.22


In [16]:
# Cargar los dos archivos CSV
df_leche = pd.read_csv('/kaggle/working/precio_leche_2013_2024.csv')   # archivo principal con precios por litro
df_ipc = pd.read_csv('/kaggle/working/ipc_2013_2024.csv')       # archivo con datos de IPC

In [17]:
# Filtrar todas las filas donde el nombre del municipio sea 'yarumal'
leche_yarumal = df_leche[df_leche['nombre municipio'].str.lower() == 'yarumal']

In [18]:
# --- Asegurar que ambos datasets tengan las columnas clave con el mismo tipo ---
leche_yarumal['año'] = leche_yarumal['año'].astype(int)
leche_yarumal['mes'] = leche_yarumal['mes'].astype(int)

df_ipc['año'] = df_ipc['año'].astype(int)
df_ipc['mes'] = df_ipc['mes'].astype(int)

# --- Hacer el merge (unión) por año y mes ---
leche_ipc = pd.merge(
    leche_yarumal,
    df_ipc,
    on=['año', 'mes'],
    how='left'   # usa 'left' para mantener todos los registros de leche
)

# --- Verificar ---
print(leche_ipc.head())
print(leche_ipc.tail())
print("\nNúmero de filas:", len(leche_ipc))
print("\nColumnas resultantes:", leche_ipc.columns.tolist())

    año  mes nombre departamento  código departamento nombre municipio  \
0  2013    1           antioquia                    5          yarumal   
1  2013    2           antioquia                    5          yarumal   
2  2013    3           antioquia                    5          yarumal   
3  2013    4           antioquia                    5          yarumal   
4  2013    5           antioquia                    5          yarumal   

   código municipio  precio promedio por litro       ipc  
0              5887                 853.410677  112.1490  
1              5887                 865.888855  112.6471  
2              5887                 844.620447  112.8788  
3              5887                 875.421163  113.1643  
4              5887                 847.117207  113.4797  
      año  mes nombre departamento  código departamento nombre municipio  \
139  2024    8           antioquia                    5          yarumal   
140  2024    9           antioquia               

/tmp/ipykernel_39/3319902926.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  leche_yarumal['año'] = leche_yarumal['año'].astype(int)
/tmp/ipykernel_39/3319902926.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  leche_yarumal['mes'] = leche_yarumal['mes'].astype(int)
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py

In [19]:
# Verificar valores nulos en todo el DataFrame
print("🔍 Revisión de valores nulos por columna:\n")
print(leche_ipc.isna().sum())

# Mostrar solo las filas que tengan al menos un valor nulo (si quieres inspeccionarlas)
filas_nulas = leche_ipc[leche_ipc.isna().any(axis=1)]
print("\n🧩 Filas con valores nulos:")
print(filas_nulas)


🔍 Revisión de valores nulos por columna:

año                           0
mes                           0
nombre departamento           0
código departamento           0
nombre municipio              0
código municipio              0
precio promedio por litro     0
ipc                          11
dtype: int64

🧩 Filas con valores nulos:
      año  mes nombre departamento  código departamento nombre municipio  \
101  2021   10           antioquia                    5          yarumal   
104  2022    1           antioquia                    5          yarumal   
105  2022    2           antioquia                    5          yarumal   
117  2022   10           antioquia                    5          yarumal   
118  2022   11           antioquia                    5          yarumal   
120  2023    1           antioquia                    5          yarumal   
133  2024    2           antioquia                    5          yarumal   
134  2024    3           antioquia                   

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


In [20]:
# --- Interpolar valores faltantes del IPC ---
leche_ipc['ipc'] = leche_ipc['ipc'].interpolate(method='linear')

# --- Verificar nuevamente ---
print("🔁 Revisión posterior a la interpolación:\n")
print(leche_ipc.isna().sum())

# (opcional) Mostrar las filas que antes estaban nulas, ya con sus nuevos valores
print("\n🧩 Filas interpoladas:")
print(leche_ipc.loc[[101, 104, 105, 117, 118, 120, 133, 134, 135, 136, 140], ['año', 'mes', 'ipc']])


🔁 Revisión posterior a la interpolación:

año                          0
mes                          0
nombre departamento          0
código departamento          0
nombre municipio             0
código municipio             0
precio promedio por litro    0
ipc                          0
dtype: int64

🧩 Filas interpoladas:
      año  mes         ipc
101  2021   10  109.840000
104  2022    1  107.066667
105  2022    2  103.533333
117  2022   10  122.486667
118  2022   11  123.473333
120  2023    1  126.365000
133  2024    2  138.760000
134  2024    3  139.800000
135  2024    4  140.840000
136  2024    5  141.880000
140  2024    9  143.845000


In [21]:
# --- Guardar el dataset limpio y completo ---
leche_ipc.to_csv('leche_yarumal_ipc_limpio.csv', index=False, encoding='utf-8')

print("✅ Archivo guardado correctamente como 'leche_yarumal_ipc_limpio.csv'")
print("📂 Número de filas:", len(leche_ipc))
print("📊 Columnas:", leche_ipc.columns.tolist())


✅ Archivo guardado correctamente como 'leche_yarumal_ipc_limpio.csv'
📂 Número de filas: 144
📊 Columnas: ['año', 'mes', 'nombre departamento', 'código departamento', 'nombre municipio', 'código municipio', 'precio promedio por litro', 'ipc']
